# Setup

In [1]:
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import wandb
from utilities import *
from config import *
from dataloading import *
from tqdm import tqdm
from transformer import *
import os

Loading data...


Loading data: 100%|██████████| 100000/100000 [00:00<00:00, 205979.55it/s]


In [ ]:
for x in train_dataloader:
    data, train = x
    break

In [ ]:
data[:10]

In [ ]:
train[:10]

In [ ]:
model = BigramLanguageModel()

In [2]:
# setup the model
model = BigramLanguageModel()

# cuda? (gpu)
if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"

# send to gpu (maybe)
model = nn.DataParallel(model)
model = model.to(device)

# optionally: load the model
filename = PATH + "/model/" + MODELNAME + ".pth"
if os.path.isfile(filename):
    model.load_state_dict(torch.load(filename, map_location=torch.device(device)))

os.path.isfile(filename)

True

In [3]:
from sklearn.decomposition import PCA
import numpy as np

embedding = np.array(model.module.token_embedding_table.weight.cpu().detach().numpy())
pos_embedding = np.array(model.module.position_embedding.weight.cpu().detach().numpy())

pca = PCA(n_components=3)
pca.fit(embedding)

embedding_pca = pca.transform(embedding)

pca = PCA(n_components=2)
pca.fit(pos_embedding)

pos_embedding_pca = pca.transform(pos_embedding)

In [11]:
import plotly.express as px

similarity = []

for x in pos_embedding:
    row = []
    for y in pos_embedding:
        row.append(np.dot(x, y))
    similarity.append(row)

px.imshow(similarity)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>color: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'xaxis': 'x',
              'yaxis': 'y',
              'z': array([[ 3.1484894e+02,  3.1861057e+01, -1.1656543e+01, -1.1272593e+01,
                           -1.5115310e+01, -2.1435186e+01,  1.0542701e+01, -3.3846378e-01,
                           -3.0878769e+01, -1.2345324e+01, -1.0355307e+01],
                          [ 3.1861057e+01,  3.5853729e+02,  1.1161618e+01, -1.6454737e+01,
                            5.1445045e+00, -5.6384101e+00, -3.5914490e+01, -2.2288975e+01,
                           -3.0223522e+00,  1.3178713e+01,  2.7697067e+00],
                          [-1.1656543e+01,  1.1161618e+01,  3.1903833e+02,  1.6294010e+01,
                           -6.2849903e-01, -1.9366959e+01, -9.5582485e-02, -1.2987326e+01,
                            1.7931557e+01,  2.7141509e+00,  8.8150902e+00],
                          [-1.1272593e+01, -1.6454737e+01,  1.6294010e+01,  3.6183746e+02,
                           -1.7901119e+01, -1.2599461e+01, -2.2391459e+01,  4.3386826e+01,
                            3.2529350e+01,  3.3441806e-01, -4.5222149e+00],
                          [-1.5115310e+01,  5.1445045e+00, -6.2849903e-01, -1.7901119e+01,
                            3.9088733e+02, -6.6800270e+00, -6.4603953e+00,  8.1313248e+00,
                            2.0957195e+01, -2.7436234e+01, -1.7595032e+01],
                          [-2.1435186e+01, -5.6384101e+00, -1.9366959e+01, -1.2599461e+01,
                           -6.6800270e+00,  3.9133234e+02, -1.6645077e+01, -9.4377871e+00,
                           -9.1309700e+00,  5.0590529e+00,  1.0106737e+01],
                          [ 1.0542701e+01, -3.5914490e+01, -9.5582485e-02, -2.2391459e+01,
                           -6.4603953e+00, -1.6645077e+01,  3.4145306e+02, -1.2534175e+01,
                            4.9710770e+00,  6.1832409e+00,  7.8097272e+00],
                          [-3.3846378e-01, -2.2288975e+01, -1.2987326e+01,  4.3386826e+01,
                            8.1313248e+00, -9.4377871e+00, -1.2534175e+01,  3.4503839e+02,
                           -2.0857124e+00,  7.9415798e-02, -3.2574249e+01],
                          [-3.0878769e+01, -3.0223522e+00,  1.7931557e+01,  3.2529350e+01,
                            2.0957195e+01, -9.1309700e+00,  4.9710770e+00, -2.0857124e+00,
                            3.6979916e+02,  7.7847977e+00,  1.4504649e+01],
                          [-1.2345324e+01,  1.3178713e+01,  2.7141509e+00,  3.3441806e-01,
                           -2.7436234e+01,  5.0590529e+00,  6.1832409e+00,  7.9415798e-02,
                            7.7847977e+00,  3.3704871e+02,  1.1680017e+00],
                          [-1.0355307e+01,  2.7697067e+00,  8.8150902e+00, -4.5222149e+00,
                           -1.7595032e+01,  1.0106737e+01,  7.8097272e+00, -3.2574249e+01,
                            1.4504649e+01,  1.1680017e+00,  3.4586542e+02]], dtype=float32)}],
    'layout': {'coloraxis': {'colorscale': [[0.0, '#0d0887'], [0.1111111111111111,
                                            '#46039f'], [0.2222222222222222,
                                            '#7201a8'], [0.3333333333333333,
                                            '#9c179e'], [0.4444444444444444,
                                            '#bd3786'], [0.5555555555555556,
                                            '#d8576b'], [0.6666666666666666,
                                            '#ed7953'], [0.7777777777777778,
                                            '#fb9f3a'], [0.8888888888888888,
                                            '#fdca26'], [1.0, '#f0f921']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis

In [9]:
px.imshow(model.module.position_embedding.weight.detach())

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>color: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'xaxis': 'x',
              'yaxis': 'y',
              'z': array([[ 1.914832  , -1.2360442 ,  1.2223556 , ..., -1.0849441 ,  1.227716  ,
                            0.53909975],
                          [ 0.6143235 , -0.7424532 , -0.6996549 , ..., -0.15660205, -0.28132066,
                           -0.21762615],
                          [-0.04699843, -0.13408513,  0.26652864, ..., -0.1192628 ,  0.12295713,
                           -1.0685372 ],
                          ...,
                          [-0.07745381,  0.25001627,  0.20818111, ..., -1.3179705 , -1.2257396 ,
                            0.1038308 ],
                          [-0.9272406 ,  0.08553367, -0.19697148, ..., -0.38630018, -1.8786055 ,
                           -0.7436882 ],
                          [ 1.2517266 ,  1.5574077 ,  0.13048102, ..., -1.1664678 ,  0.21812972,
                           -0.2316184 ]], dtype=float32)}],
    'layout': {'coloraxis': {'colorscale': [[0.0, '#0d0887'], [0.1111111111111111,
                                            '#46039f'], [0.2222222222222222,
                                            '#7201a8'], [0.3333333333333333,
                                            '#9c179e'], [0.4444444444444444,
                                            '#bd3786'], [0.5555555555555556,
                                            '#d8576b'], [0.6666666666666666,
                                            '#ed7953'], [0.7777777777777778,
                                            '#fb9f3a'], [0.8888888888888888,
                                            '#fdca26'], [1.0, '#f0f921']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})

In [ ]:
torch.no_grad()
model.eval()

In [ ]:
targets

In [ ]:
# calculate validation stats
criterion = nn.CrossEntropyLoss()

total_accuracy = 0.0
total_loss = 0.0

num_batches = 0

print("Evaluating...")
for inputs, targets in tqdm(val_dataloader):
    outputs = model(inputs)

    # calculate the val accuracy
    accuracy = calculate_accuracy(outputs, targets)
    total_accuracy += accuracy

    # Calculate the val loss
    loss = criterion(outputs, targets)
    total_loss += loss.item()
    num_batches += 1

average_accuracy = total_accuracy / num_batches
val_loss = total_loss / num_batches

In [ ]:
val_loss

In [ ]:
vocab_size

In [ ]:
output = model(data)

In [ ]:
train

In [ ]:
train.shape

In [ ]:
output.shape

In [ ]:
output[1]

In [ ]:
model.get_device()

In [ ]:
dev

In [ ]:
if cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

In [ ]:
conver

In [ ]:
seq = [0,0,2,20,1,0]
get_permutation(seq)

In [ ]:
model.module.generate(seq)

In [ ]:
model.eval()

# use gpu for processing
if cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

# create an initial input
input_tensor = torch.ones(block_size, dtype=int).to(dev)
input_tensor *= TO_PREDICT_TOKEN
input_tensor[:len(seq)] = torch.tensor(seq, dtype=int).to(dev)
input_tensor[len(seq)] = START_PREDICTION_TOKEN

In [ ]:
input_tensor.unsqueeze(0)

In [ ]:
model(input_tensor.unsqueeze(0))

In [ ]:
torch.argmax(model(input_tensor.unsqueeze(0)), dim=1)

In [ ]:
prediction_tensor = torch.zeros(block_size, dtype=int).to(dev)

In [ ]:
sequence = [1,2,3,1,2,3]

In [ ]:
prediction_tensor[:len(sequence)] = torch.tensor(sequence, dtype=int).to(dev)

In [ ]:
prediction_tensor

In [ ]:
input_tensor = torch.ones(block_size, dtype=int).to(dev)

In [ ]:
input_tensor *= TO_PREDICT_TOKEN

In [ ]:
input_tensor